In [1]:
import os
os.chdir('../')
os.getcwd()
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings.local")

'config.settings.local'

In [125]:
os.chdir(r"D:\workspace\thetatauCMT\scratchpad\2020Update")

In [2]:
import django
from django_pandas.io import read_frame
import plotly_express as px
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
django.setup()

In [4]:
import plotly
import chart_studio.plotly as py
from plotly import tools
import plotly.graph_objs as go

In [111]:
from chapters.models import Chapter
from scores.models import ScoreType
from regions.models import Region
from forms.models import Depledge
from django.db import models
from core import models as core_models
from events.models import Event
from forms.models import Initiation, StatusChange
from tasks.models import TaskChapter
from users.models import UserRoleChange, UserOrgParticipate, UserSemesterServiceHours, UserSemesterGPA

In [6]:
test = Chapter.objects.first()

In [7]:
test.actives().count()

28

In [8]:
test.members.exclude(models.Q(depledge__isnull=True)).first().depledge.reason

'volunteer'

In [9]:
colors = {
    'actives': 'darkred',
    'pledges': 'gold',
    'events': 'green',
    'submissions': 'yellow',
    'Bro': 'darkred',
    'Ops': 'skyblue',
    'Pro': 'gold',
    'Ser': 'magenta',
    'volunteer': 'darkred',
    'time': 'skyblue',
    'grades': 'gold',
    'interest': 'magenta',
    'vote': 'orange',
    'withdrew': 'grey',
    'transfer': 'turquoise',
    'other': 'turquoise',
}

In [11]:
test_region = Region.objects.last()

In [12]:
test_region.chapters.all()

<QuerySet [<Chapter: Epsilon>, <Chapter: Lambda Delta>, <Chapter: Mu Delta>, <Chapter: Omicron Gamma>, <Chapter: Rho Delta>, <Chapter: SCU Colony>, <Chapter: SJSU Colony>, <Chapter: UW Colony>]>

# Graph current pledges/actives each chapter

In [126]:
DataOut=[]
legend=True
for chapter in Chapter.objects.all():
    if chapter.name == "TEST" or not chapter.active:
        continue
    trace= go.Bar(
        x=[chapter.name],
        y=[chapter.actives().count()],
        visible=True,
        name=f"Actives",
        marker=dict(color=colors['actives']),
        showlegend=legend,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[chapter.name],
        y=[chapter.pledges().count()],
        visible=True,
        name=f"Pledges",
        marker=dict(color=colors['pledges']),
        showlegend=legend,
    )
    legend=False
    DataOut.append(trace)

In [127]:
menu = [
    dict(label = "ALL",
         method = 'update',
         args = [{'visible': [True for data_obj in DataOut]},
                 ]),
       ]
for region in Region.objects.all().order_by('name'):
    if region.name == "Test":
        continue
    chapter_names = region.chapters.all().values_list('name', flat=True)
    menu_obj = dict(label = region.name,
         method = 'update',
         args = [{'visible': [True if data_obj['x'][0] in chapter_names else False for data_obj in DataOut]},
                 ])
    menu.append(menu_obj)
updatemenus = list([
    dict(active=0,
         buttons=menu,
    )
])

In [128]:
layout = go.Layout(
    barmode='stack',
    updatemenus=updatemenus,
    title="Current Active vs Pledges Per Chapter"
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="chapter_activeVSpledges.html")

'chapter_activeVSpledges.html'

# Events for each chapter by type

In [129]:
for i, (date_name, start_end)  in enumerate(core_models.BIENNIUM_DATES.items()):
    DataOut=[]
    legend=True
    for chapter in Chapter.objects.all():
        if chapter.name == "TEST" or not chapter.active:
            continue
        for evt_type in [x.value[0] for x in ScoreType.SECTION]:
            count = chapter.events.filter(
                date__lte=start_end['end'],
                date__gte=start_end['start']
            ).filter(type__section=evt_type).count()
            trace= go.Bar(
                x=[chapter.name],
                y=[count],
                visible=True,
                name=f"{evt_type}",
                marker=dict(color=colors[evt_type]),
                showlegend=legend,
            )
            DataOut.append(trace)
        legend=False
    layout = go.Layout(
        barmode='stack',
        updatemenus=updatemenus,
        title=f"{date_name} Events per Chapter by Event Type",
    )
    fig = go.Figure(data=DataOut, layout=layout)
    plotly.offline.plot(fig, filename=f"chapter_eventTypes_{date_name}.html")

# Depledge Reason by Chapter

In [133]:
DataOut=[]
legend=True
for chapter in Chapter.objects.all():
    if chapter.name == "TEST" or not chapter.active:
        continue
    depledge_count = 0
    for type_value in [x.value[0] for x in Depledge.REASONS]:
        count = chapter.members.exclude(
            models.Q(depledge__isnull=True)).filter(
            depledge__reason=type_value, depledge__date__gt=core_models.BIENNIUM_DATES['Fall 2019']['start']).count()
        depledge_count += count
        trace= go.Bar(
            x=[chapter.name],
            y=[count],
            visible=True,
            name=f"{type_value}",
            marker=dict(color=colors[type_value]),
            showlegend=legend,
        )
        DataOut.append(trace)
    pledges = chapter.members.filter(
            status__status="pnm",
            status__start__gte=core_models.BIENNIUM_DATES['Fall 2019']['start'],
            status__end__lte=core_models.BIENNIUM_DATES['Fall 2019']['end']).count() - depledge_count
    trace= go.Bar(
        x=[chapter.name],
        y=[pledges],
        visible=True,
        name=f"Total Pledges",
        marker=dict(color='red'),
        showlegend=legend,
    )
    DataOut.append(trace)
    legend = False
layout = go.Layout(
    barmode='stack',
    updatemenus=updatemenus,
    title="Fall 2019 Depledge Reason per Chapter"
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="chapter_depledges.html")

'chapter_depledges.html'

# Total Counts

In [152]:
DataOut=[]
for i, (date_name, start_end)  in enumerate(core_models.BIENNIUM_DATES.items()):
    trace= go.Bar(
        x=[f"Events"],
        y=[Event.objects.filter(
            date__lte=start_end['end'],
            date__gte=start_end['start']).count()],
        visible=True,
        name=f"Events {date_name}",
        hoverlabel={
            'namelength':-1,
        },
        marker=dict(color='darkred'),
        showlegend=False,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Initiations"],
        y=[Initiation.objects.filter(
            date__lte=start_end['end'],
            date__gte=start_end['start']).count()],
        visible=True,
        name=f"Initiations {date_name}",
        hoverlabel={
            'namelength':-1,
        },
        marker=dict(color='gold'),
        showlegend=False,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Status Change"],
        y=[StatusChange.objects.filter(
            date_start__lte=start_end['end'],
            date_start__gte=start_end['start']).count()],
        visible=True,
        name=f"Status Change {date_name}",
        hoverlabel={
            'namelength':-1,
        },
        marker=dict(color='magenta'),
        showlegend=False,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Officer Reports"],
        y=[UserRoleChange.objects.filter(
            start__lte=start_end['end'],
            start__gte=start_end['start']).count()],
        visible=True,
        name=f"Officer Reports {date_name}",
        hoverlabel={
            'namelength':-1,
        },
        marker=dict(color='cyan'),
        showlegend=False,
    )
    DataOut.append(trace)
layout = go.Layout(
    title="Total Biennium Counts"
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="Counts.html")

'Counts.html'

In [30]:
UserOrgParticipate.objects.all().count()

2357

In [54]:
all_orgs = UserOrgParticipate.objects.filter(
            start__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
            end__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).values_list('org_name', flat=True)

In [55]:
all_orgs.count()

181

In [58]:
orgs_sep = []
for org in all_orgs:
    orgs_sep.extend(org.split(","))

In [59]:
orgs_cleaned = [org.strip() for org in orgs_sep]

In [135]:
len(orgs_cleaned)

182

In [60]:
from collections import Counter

In [61]:
count = Counter(orgs_cleaned)

In [63]:
count.most_common(10)

[('ASME', 9),
 ('IEEE', 8),
 ('BMES', 7),
 ('Tau Beta Pi', 5),
 ('SWE', 5),
 ('ASCE', 4),
 ('AIChE', 4),
 ('Alpha Phi Omega', 3),
 ('American Society of Civil Engineers', 3),
 ('Club Soccer', 3)]

In [65]:
UserSemesterServiceHours.objects.all().count()

4683

In [107]:
total = 0
print("Semester", 'Service Total', sep='\t')
for i, (date_name, start_end)  in enumerate(core_models.BIENNIUM_DATES.items()):
    semester, year = date_name.split(' ')
    sem = {'Fall': 'fa', 'Spring': 'sp'}[semester]
    all_service = UserSemesterServiceHours.objects.filter(
                year=int(year), term=sem,).values_list('service_hours', flat=True)
    sem_total = sum(all_service)
    total += sem_total
    print(date_name, sem_total, sep='\t')
print("Total Bi", total, sep='\t')

Semester	Service Total
Fall 2018	3963.6
Spring 2019	4738.75
Fall 2019	2706.5
Spring 2020	51.0
Total Bi	11459.85


In [110]:
all_service = UserSemesterServiceHours.objects.filter(
                year=2018, term='fa',).values_list('service_hours', flat=True) | \
UserSemesterServiceHours.objects.filter(
            year__in=[2019, 2020]).values_list('service_hours', flat=True)
print(all_service.count())
print(sum(all_service) / all_service.count())

1197
9.573809523809524


In [112]:
all_gpa = UserSemesterGPA.objects.filter(
                year=2018, term='fa',).values_list('gpa', flat=True) | \
UserSemesterGPA.objects.filter(
            year__in=[2019, 2020]).values_list('gpa', flat=True)
print(all_gpa.count())
print(sum(all_gpa) / all_gpa.count())

2571
3.343719175418123


In [113]:
max(all_gpa)

4.2

In [114]:
min(all_gpa)

0.99

In [137]:
Event.objects.filter(date__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                     date__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

4808

In [138]:
Submission.objects.filter(date__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                          date__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

446

In [139]:
from tasks.models import TaskChapter

In [140]:
TaskChapter.objects.filter(date__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                           date__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

2377

In [142]:
Initiation.objects.filter(date__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                          date__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

2245

In [151]:
StatusChange.objects.filter(date_start__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                            date_start__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

2369

In [145]:
from forms.models import Audit

In [146]:
Audit.objects.filter(created__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                     created__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

d:\workspace\.virtualenvs\cmt\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning:

DateTimeField Audit.created received a naive datetime (2018-07-01 00:00:00) while time zone support is active.

d:\workspace\.virtualenvs\cmt\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning:

DateTimeField Audit.created received a naive datetime (2020-06-30 00:00:00) while time zone support is active.



363

In [150]:
Depledge.objects.filter(date__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                        date__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

432

In [149]:
UserRoleChange.objects.filter(start__gte=core_models.BIENNIUM_DATES['Fall 2018']['start'],
                              start__lte=core_models.BIENNIUM_DATES['Spring 2020']['end']).count()

2277